## Lab 9: Camera calibration

Written by: Enrique Mireles Gutiérrez  
ID Number: 513944  
Bachelor: ITR  
Date: 2019-04-22  

### Introduction

Camera calibration is the process of estimating intrinsic and/or extrinsic parameters of the camera. These parameters describe the camera and its distortions. Intrinsic parameters are those that have to do with internal characteristics of the camera, such as the focal length, distortion, and the image center. Extrinsic parameters have to do with the position and orientation of the camera in space.  

This lab report focuses on performing calibration for intrinsic parameters. Since no camera is built perfectly, the images obtained have distortions, also known as radial and tangential distortions. Therefore, by knowing how the image is distorted it is then possible to compensate for it and thus increase the accuracy of the camera. In applications, such as autonomous vehicles, calibrating the camera is essential.

In order to calibrate the camera, several images of a standard calibration pattern are taken and then processed by optimization methods found in OpenCV in order to estimate the intrinsic parameters of the camera. Different types calibration patterns exist for this purpose. Nonetheless, a checkerboard pattern was chosen for its simplicity and effectiveness in this lab report.

### Objectives

This lab has the following objectives:
- Design and print a checkerboard pattern for calibration.
- Take images from the checkerboard pattern and use them to calibrate the camera.
- Use the obtained calibration to undistort images.

### Procedure

This lab report is subdivided in smaller numbered tasks shown below.

#### 1. Printing and preparing the calibration pattern

A checkerboard pattern was chosen for the calibration process. The pattern was generated using the website https://calib.io/pages/camera-calibration-pattern-generator. The settings used to generate the pattern were:
- Target Type: Checkerboard
- Board Width [mm]: 215.9 (8.5 in)
- Board Height [mm]: 279.4 (11 in)
- Rows: 11
- Columns: 8
- Checker Width [mm]: 20

The board sizes where chosen such that they match a standard letter size paper. Furthermore, the amount of rows and columns was chosen to be odd and even respectively. The important part is that they aren’t both even or odd. By using the proposed values, a well sized checkerboard pattern could be printed on a standard letter size paper. The printer was configured to print as dark as possible and deactivated any scaling done to the printed image. 

The width of the checkers was then validated using digital calipers. The width of the checkers was actually 20mm. Finally, a glass sheet was used as a flat surface for placing the pattern. It was taped at the top and heavy books were left lying on top of the calibration pattern. After some hours, without lifting the paper, the calibration pattern was taped from all edges. Doing so left a flat pattern taped to the glass sheet. The calibration pattern was at this moment ready for its use.

#### 2. The camera and its focus

The camera used was a Raspberry Pi Camera Module V2 (https://www.raspberrypi.org/products/camera-module-v2/) This module, as its name implies, runs on the Raspberry Pi and it will be the camera that runs on the autonomous vehicle. The camera comes with a fixed focus at infinity. However, by testing the camera it was shown that somehow the glue that holds the lens had stopped working and caused the lens to move out of focus. The camera was manually focused by turning the lens. It’s important that the images taken for calibration are in focus, otherwise the algorithms that analyze the image won’t find the pattern.

#### 3. Importing Libraries

The following libraries are used throughout the lab report:
- cv2: OpenCV library used for artificial vision.
- numpy: library used for matrix operations.
- os: library used for operating system interfaces.
- datetime: library used for getting timestamps.

In [1]:
import cv2
import numpy as np
import os
import datetime

#### 4. Constant definitions

The following lines define the constants used throughout the lab report:

In [2]:
IMAGE_FOLDER = os.getcwd() + '/original/'
CHESSBOARD_COLS = 10
CHESSBOARD_ROWS = 7
CHESSBOARD_CELL_WIDTH = 20
CHESSBOARD_DIMENSIONS = (CHESSBOARD_COLS, CHESSBOARD_ROWS)
FRAME_WIDTH = 1280
FRAME_HEIGHT = 720
FRAME_DIMENSIONS = (FRAME_WIDTH, FRAME_HEIGHT)

# Get all images from source directory.
IMAGES = []
for r, d, f in os.walk(IMAGE_FOLDER):
    for file in f:
        IMAGES.append(IMAGE_FOLDER + file)

#### 5. Taking calibration images

A script was made that allowed a user to save pictures when the space-bar was pressed. Each picture was saved to an output folder, and with the help of an assistant, 31 different pictures were taken. Extra care was made to try to reduce glare on the calibration pattern (closing window blinds, turning on lamps, etc.). The position and orientation of the calibration pattern was changed in all directions and different -z depths were tested. All of this was done in order to obtain better calibration results.

It's important to note that the code from this section requires a Raspberry Pi camera to run properly.

In [ ]:
from picamera.array import PiRGBArray
from picamera import PiCamera
import time
import datetime

# Open raspberry pi's camera.
camera = PiCamera()
camera.resolution = FRAME_DIMENSIONS
camera.framerate = 24
rawCapture = PiRGBArray(camera, size=FRAME_DIMENSIONS)

# Wait some time for it to init.
time.sleep(0.1)

# Counter for amount of images saved.
image_count = 0

# While loop - pi camera version.
for frame in camera.capture_continuous(rawCapture, format="bgr", use_video_port=True):
    
    # Get image.
    image = frame.array

    # Display current image.
    cv2.imshow("Frame", image)

    # Wait for a key press and get next frame.
    key = cv2.waitKey(1) & 0xFF
    rawCapture.truncate(0)

    # Check letter pressed.
    if key == ord('q'):
        break
    elif key == ord(' '):
        # If space is pressed, then get the current time and generate an output file name.
        time = str(datetime.datetime.now()).split('.')[0]
        output_filename = './output/' + time + '.jpg'

        # Write image and log to command window.
        cv2.imwrite(output_filename, image)
        image_count += 1
        print(str(image_count) + ' File written at: ' + output_filename)

# Close all windows.    
cv2.destroyAllWindows()

The images obtained in this section were compiled into a single .gif file. This was done with help from the popular image manipulation program Imagemagick. By going into the output folder and running the following command, all .jpg images in the current folder were compiled into a .gif:

`convert -delay 30 -loop 0 *.jpg original.gif`

This is the result:

<img src="original.gif" width="400" alt="Original images.">

#### 6. Camera calibration

The following program takes the images from the output folder and searches for the calibration pattern. If found, the calibration pattern is then displayed over the original image and used to calibrate the camera. At first, the delay on `waitKey` was increased to 2000 ms, and the code for calibrating the camera was commented out. This allowed the user to distinguish if an image didn’t work for the calibration pattern. However, all pictures taken worked flawlessly. The calibration code was uncommented and the delay  was reduced to 50 ms. Finally, the program outputs the camera matrix, the distortion matrix, the RMS error, and the rotation and translation output vectors of the checkerboard.

In [3]:
# Init arrays for storing chessboard points.
obj_points = []
img_points = []

# Generate an empty np array with the dimensions of the chessboard.
# This array is required for calculating the camera distortion.
pattern_points = np.zeros((np.prod(CHESSBOARD_DIMENSIONS), 3), np.float32)
pattern_points[:, :2] = np.indices(CHESSBOARD_DIMENSIONS).T.reshape(-1, 2)
pattern_points *= CHESSBOARD_CELL_WIDTH

# Iterate through all images.
for file in IMAGES:

    # Read input image and convert to grayscale.
    img = cv2.imread(file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY);

    # Find chess board corners.
    ret, corners = cv2.findChessboardCorners(gray, CHESSBOARD_DIMENSIONS, None)

    # If chess board is found:
    if (ret):
        
        # Find corners with better resolution.
        corners2 = cv2.cornerSubPix(
            gray, 
            corners, 
            (11, 11), 
            (-1, -1), 
            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, CHESSBOARD_CELL_WIDTH, 0.001)
        )
        
        # Draw the found corners on the input image.
        cv2.drawChessboardCorners(img, CHESSBOARD_DIMENSIONS, corners2, ret)
        
        # Saved the found data to an array.
        img_points.append(corners2)
        obj_points.append(pattern_points)
        
        # Uncomment to save output file.
        # cv2.imwrite(file.replace('.', '-corners.'), img)
        
    else:
        print('Chessboard not found: ' + file)

    # Display image and wait 50 ms between frames.
    cv2.imshow('input', img)
    cv2.waitKey(50)

# Calculate camera distortion.
RMS, CAMERA_MATRIX, DISTORTION_COEFFICIENTS, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, FRAME_DIMENSIONS, None, None)

# Display results.
print('RMS error', RMS)
print('Camera matrix', CAMERA_MATRIX)
print('Distortion coefficients', DISTORTION_COEFFICIENTS)
print('Output vector of rotation vectors', rvecs)
print('Output vector of translation vectors', tvecs)

# Close all windows.    
cv2.destroyAllWindows()

RMS error 0.17605950343351903
Camera matrix [[1.00612323e+03 0.00000000e+00 6.31540281e+02]
 [0.00000000e+00 1.00551440e+03 3.48207362e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients [[ 0.18541226 -0.32660915  0.00088513 -0.00038131 -0.02052374]]
Output vector of rotation vectors [array([[ 0.34330688],
       [-0.02407662],
       [-1.6211368 ]]), array([[0.60826344],
       [0.08193414],
       [0.89317572]]), array([[ 0.28250445],
       [-0.27637923],
       [ 0.06208262]]), array([[-0.12331523],
       [-0.38790127],
       [-1.50296207]]), array([[ 0.11622997],
       [-0.23183826],
       [ 0.01983189]]), array([[ 0.16194306],
       [ 0.15633236],
       [-0.0125181 ]]), array([[-0.1688296 ],
       [-0.58221824],
       [ 0.70511442]]), array([[-0.18227042],
       [-0.32888829],
       [ 0.58915959]]), array([[ 0.04879359],
       [-0.60104839],
       [ 0.02237523]]), array([[ 0.35217944],
       [-0.62736339],
       [-1.35494189]]), array([[ 0.

Saving the output images from this step and compiling them into a single .gif yielded the following results:

<img src="chessboard-corners.gif" width="400">

#### 7. Undistort images

The following program takes the calibration and distortion matrix from the last section, and uses them to undistort the images taken.

In [4]:
# Window definitions.
cv2.namedWindow('raw', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('undistorted', cv2.WINDOW_AUTOSIZE)

# Iterate through all images.
for file in IMAGES:

    # Read input image.
    img = cv2.imread(file)

    # Generate the new camera matrix
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(CAMERA_MATRIX, DISTORTION_COEFFICIENTS, FRAME_DIMENSIONS, 1, FRAME_DIMENSIONS)
    
    # Undistort and display images.
    dst = cv2.undistort(img, CAMERA_MATRIX, DISTORTION_COEFFICIENTS, None, newcameramtx)
    cv2.imshow('raw', img)
    cv2.imshow('undistorted', dst)
    
    # Uncomment to save undistorted images.
    # cv2.imwrite(file.replace('.', '-undistorted.'), dst)
    
    # Delay between images.
    cv2.waitKey(100)

# Close all windows.
cv2.destroyAllWindows()

Saving the output images from this step and compiling them into a single .gif yielded the following results:

<img src="chessboard-corners-undistorted.gif" width="400">

### Conclusions

By analyzing the results obtained in section 6, one can see that the calibration’s RMS error is around 0.176px, which is better than acceptable range set at 0.25px. This means that the calibration was done successfully and that it can be used for further applications. Moreover, it’s important to note that the center of the image doesn’t exactly lie on the image’s center. It is about 10px to the left, and 10px to the top from the theoretical center. The reasons behind this could be caused by the amount of distortion found at the edges and thus shift the center in a top-left direction.

I’ve come to the conclusion that the amount of effort that one puts into the calibration process is inversely proportional to the RMS error. More pictures taken could yield better results. Also better lighting conditions or even a flatter calibration patter would increase accuracy. Nonetheless, by following simple techniques, such as flattening the calibration pattern with books, avoiding reflections on the image, and testing different orientations and positions of the pattern, produced good results on the first run.

The following gifs toggle between the distorted and undistorted images. This allows the user to see how the image is changing and if distortions are properly corrected.

<img src="animation.gif">
<img src="animation-2.gif">

### References
- https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_calibration/py_calibration.html
- https://boofcv.org/index.php?title=Tutorial_Camera_Calibration

_I hereby affirm that I have done this activity with academic integrity._